In [1]:
import numpy as np
from pint import UnitRegistry
import math
ureg = UnitRegistry()
np.set_printoptions(precision=4)
import sys
sys.path.append('/Users/Lampe/PyScripts/surgeAnalysis')
import SurgeFunc as sf

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

### Problem Data

In [6]:
# numerical parameters
nel = int(4) # number of elements
tf = 10.0 # total calculation tim (sec)

#problem Data
Tc = 7. # time for total valve closure (sec)
degTheta = 90.0 # horizontal well
p0 = 0.6638662 # reservoir pressure (MPa)
q0 = 1.0*3600 # SS flow rate (m3/hr)

mu = np.array([1.60 * 10**-3,1.60 * 10**-3]) # Dynamic viscosity (Pa-s, N-s/m2)
sg = np.array([1.0, 1.0]) # specific gravity
rough = np.array([0.9, 0.9]) # roughness (mm)
d = np.array([750., 600.]) # mm
a = np.array([1100., 900.]) # speed of sound in fluid (m/s)
L = np.array([550., 450.]) # length of each conduit reach (m)

### Calculated Constants

In [7]:
nnode = nel + 1 # number of nodes
x = np.linspace(0., sum(L), nnode) # spatial discretization
nrlp = np.sum(x[:]>=L[-1])# number of reachs in last segment

# dx = sum(L)/nel
x = np.array([0,550,1000]) #forced spatial discretization
s = L[-1]/(2*a[-1])#dx / max(a); # time step size (s)
t = np.linspace(0, tf, tf/s + 1) # time discretization
tau = sf.valvePercOpen(t, Tc)

array([ 1.    ,  0.9942,  0.9785,  0.9545,  0.9236,  0.8874,  0.8473,
        0.8044,  0.7599,  0.7149,  0.6705,  0.6271,  0.5843,  0.5421,
        0.5   ,  0.4579,  0.4157,  0.3729,  0.3295,  0.2851,  0.2401,
        0.1956,  0.1527,  0.1126,  0.0764,  0.0455,  0.0215,  0.0058,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ])

### Fluid / Conduit specific values

In [10]:
A = np.pi * (d / 1000.)**2 / 4. # area of flow (m2)
ani = L / (s*a)
g = 9.806 # m/s2
rho = sg * 1000 # kg/m3

f = np.zeros(len(d))
ca = np.zeros(len(d))
cf = np.zeros(len(d))
for i in xrange(len(d)):
#     f[i] = sf.SS_DeltaH(q0, mu[i], sg[i], d[i], rough[i], x, degTheta, units=1).Calc().f # friction factor 
    f[0] = 0.01
    f[1] = 0.012
    ca[i] = g * A[i] / a[i] # m2/sec
    cf[i] = s * f[i]/(2*(d[i]/1000.)*A[i]) #assumed constant (sec/m3)

### Create Empty Arrays

In [ ]:
# rows-> spatial disc; column -> temporal disc
h = np.zeros(len(x)) # SS head (m)
q = np.zeros(len(x)) # SS flow rate (m3/hr)
qp = np.zeros((len(x), len(t))) # Transient flow rate along the positive characteristic (m3/hr)
hp = np.zeros((len(x), len(t))) # Transient head along the positive characteristic (m3/hr)
p = np.zeros((len(x), len(t))) # pressure (MPa)

### Steady State Calc

In [12]:
q = q0 / 3600. # steady flow (m3/sec)
h[0,0] = 67.7 # m


# h = (p0*10**6)/(rho*g) + sf.SS_DeltaH(q0, mu, sg, d, rough, x, units=1).Calc().dh # head (m)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [7]:
# currently assumed constant - will not be constant for different fluids


# upstream reservoir
cn = q[0,1] - h[0,1]*ca - cf * q[0,1] * np.abs(q[0,1]) # negative characteristic (m3)
qp[0,0] = cn + ca*h[0,0]

# interior nodes
i_arr = np.arange(0,len(x),1) # spatial index
j_arr = np.arange(1,len(t),1) # time index
for i in i_arr:
    for j in j_arr:
        jp1 = j + 1
        jm1 = j - 1
        cn = q[i, jp1] - h[i,jp1]*ca

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])